Loading Data

In [57]:
import pandas as pd
import json

In [58]:
# Load CSV files
transactions = pd.read_csv("transactions_data.csv")
cards = pd.read_csv("cards_data.csv")
users = pd.read_csv("users_data.csv")

# Load JSON files
with open("mcc_codes.json") as f:
    mcc_codes_data = json.load(f)
mcc_codes = pd.DataFrame(list(mcc_codes_data.items()), columns=["mcc_code", "description"])
with open("train_fraud_labels.json") as f:
    fraud_labels = json.load(f)
fraud_labels = pd.DataFrame(list(fraud_labels["target"].items()), columns=["id", "isFraud"])

Merging datasets

In [59]:
fraud_labels['id'] = fraud_labels['id'].astype(int)
cards.rename(columns={'id': 'card_id'}, inplace=True)
users.rename(columns={'id': 'client_id'}, inplace=True)
mcc_codes.rename(columns={'mcc_code': 'mcc'}, inplace=True)
mcc_codes['mcc'] = mcc_codes['mcc'].astype(int)

In [60]:
# Merge transaction data with fraud labels
transactions = transactions.merge(fraud_labels, on="id", how="left")

# Merge card data
transactions = transactions.merge(cards, on=["card_id","client_id"], how="left")

# Merge user data
transactions = transactions.merge(users, on="client_id", how="left")

# Add MCC codes
transactions = transactions.merge(mcc_codes, on="mcc", how="left")

In [61]:
transactions.to_csv("merged_transactions.csv", index=False)

Investigating dataset summary & fixing missing values

In [62]:
transactions = pd.read_csv("merged_transactions.csv")
# Check general info
print(transactions.info())

# Check for missing values
print(transactions.isnull().sum())

# View the first few rows
print(transactions.head())

# Check unique values in categorical columns
print(transactions[['card_brand', 'card_type', 'gender', 'isFraud']].nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13305915 entries, 0 to 13305914
Data columns (total 38 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   id                     int64  
 1   date                   object 
 2   client_id              int64  
 3   card_id                int64  
 4   amount                 object 
 5   use_chip               object 
 6   merchant_id            int64  
 7   merchant_city          object 
 8   merchant_state         object 
 9   zip                    float64
 10  mcc                    int64  
 11  errors                 object 
 12  isFraud                object 
 13  card_brand             object 
 14  card_type              object 
 15  card_number            int64  
 16  expires                object 
 17  cvv                    int64  
 18  has_chip               object 
 19  num_cards_issued       int64  
 20  credit_limit           object 
 21  acct_open_date         object 
 22  year_pin_last_ch

In [63]:
transactions.fillna({'merchant_state':'Unknown'}, inplace=True)
transactions.fillna({'zip':'Unknown'}, inplace=True)
transactions.fillna({'errors':'No Error'}, inplace=True)

Splitting train and test dataset based on available fraud labels

In [64]:
# Split the dataset
train_data = transactions[transactions['isFraud'].notna()]
test_data = transactions[transactions['isFraud'].isna()]

# Convert 'isFraud' in train_data to numeric
train_data.loc[:,'isFraud'] = train_data.loc[:,'isFraud'].map({'Yes':1,'No':0}).astype(int)

# Drop 'isFraud' from test_data since it's unavailable
test_data.drop(columns=['isFraud'], inplace=True)

# Verify splits
print(f"Train shape: {train_data.shape}")
print(f"Test shape: {test_data.shape}")

Train shape: (8914963, 38)
Test shape: (4390952, 37)


C:\Users\kvanveen1\AppData\Local\Temp\ipykernel_10776\160459166.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.drop(columns=['isFraud'], inplace=True)


In [65]:
train_data.to_csv("train_data.csv", index=False)
test_data.to_csv("test_data.csv", index=False)

Fix data types

In [66]:
train_data = pd.read_csv("train_data.csv")
test_data = pd.read_csv("test_data.csv")

monetary_cols = ['amount', 'credit_limit', 'per_capita_income', 'yearly_income', 'total_debt']
for col in monetary_cols:
    train_data.loc[:,col] = train_data.loc[:,col].replace('[\$,]', '', regex=True).astype(float)
    test_data.loc[:,col] = test_data.loc[:,col].replace('[\$,]', '', regex=True).astype(float)

<>:6: SyntaxWarning: invalid escape sequence '\$'
<>:7: SyntaxWarning: invalid escape sequence '\$'
<>:6: SyntaxWarning: invalid escape sequence '\$'
<>:7: SyntaxWarning: invalid escape sequence '\$'
C:\Users\kvanveen1\AppData\Local\Temp\ipykernel_10776\2251030278.py:6: SyntaxWarning: invalid escape sequence '\$'
  train_data.loc[:,col] = train_data.loc[:,col].replace('[\$,]', '', regex=True).astype(float)
C:\Users\kvanveen1\AppData\Local\Temp\ipykernel_10776\2251030278.py:7: SyntaxWarning: invalid escape sequence '\$'
  test_data.loc[:,col] = test_data.loc[:,col].replace('[\$,]', '', regex=True).astype(float)


In [67]:
train_data['date'] = pd.to_datetime(train_data['date'])
test_data['date'] = pd.to_datetime(test_data['date'])

In [68]:
categorical_cols = ['use_chip', 'merchant_city', 'merchant_state', 'card_brand', 'card_type', 'gender']
for col in categorical_cols:
    train_data.loc[:,col] = train_data.loc[:,col].astype('category')
    test_data.loc[:,col] = test_data.loc[:,col].astype('category')

In [69]:
# Standardize and map for binary encoding
binary_cols = ['has_chip', 'card_on_dark_web']

for col in binary_cols:
    train_data[col] = train_data[col].str.upper().map({'YES': 1, 'NO': 0}).astype(int)
    test_data[col] = test_data[col].str.upper().map({'YES': 1, 'NO': 0}).astype(int)

Feature engineering

In [70]:
# Extract temporal features
train_data['year'] = train_data['date'].dt.year
train_data['month'] = train_data['date'].dt.month
train_data['day'] = train_data['date'].dt.day
train_data['hour'] = train_data['date'].dt.hour
train_data['day_of_week'] = train_data['date'].dt.dayofweek

test_data['year'] = test_data['date'].dt.year
test_data['month'] = test_data['date'].dt.month
test_data['day'] = test_data['date'].dt.day
test_data['hour'] = test_data['date'].dt.hour
test_data['day_of_week'] = test_data['date'].dt.dayofweek

In [71]:
# Create derived features
train_data['debt_to_income_ratio'] = train_data['total_debt'] / train_data['yearly_income']

test_data['debt_to_income_ratio'] = test_data['total_debt'] / test_data['yearly_income']

In [72]:
# Transactions per client
train_data['transactions_per_client'] = train_data.groupby('client_id')['id'].transform('count')
test_data['transactions_per_client'] = test_data.groupby('client_id')['id'].transform('count')

In [74]:
train_data.to_csv("cleaned_train.csv", index=False)
test_data.to_csv("cleaned_test.csv", index=False)
print("Dataset cleaned and saved.")

Dataset cleaned and saved.
